# Table_Image-to-CSV

Tesseract has unicode (UTF-8) support, and can recognize more than 100 languages "out of the box".This project does not include a GUI application. If you need one, please see the 3rdParty documentation.Tesseract can be trained to recognize other languages. See Tesseract Training for more information.

#### WARNING:
Tesseract should be either installed in the directory which is suggested during the installation or in a new directory. The uninstaller removes the whole installation directory. If you installed Tesseract in an existing directory, that directory will be removed with all its subdirectories and files.

The latest installers can be downloaded here:

[tesseract-ocr-w32-setup-v5.0.0-alpha.20201127.exe (32 bit)](https://digi.bib.uni-mannheim.de/tesseract/tesseract-ocr-w32-setup-v5.0.0-alpha.20201127.exe) and
[tesseract-ocr-w64-setup-v5.0.0-alpha.20201127.exe (64 bit)](https://digi.bib.uni-mannheim.de/tesseract/tesseract-ocr-w64-setup-v5.0.0-alpha.20201127.exe) resp.

##### Install Tesseract.exe from the link provided in readme file and load the pytesseract to the local machine,load the path in the installed file location

In [1]:
import pytesseract
#This is the important step,should load the path in the installed directory
pytesseract.pytesseract.tesseract_cmd=r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [2]:
import cv2
from pytesseract import Output
import pandas as pd

In [3]:
#Load the image
img = cv2.imread(r'E:\Data Science\AI_ChatBot_Python-master\image.png')
img = cv2.resize(img, (int(img.shape[1] + (img.shape[1] * .1)),
                       int(img.shape[0] + (img.shape[0] * .25))),
                 interpolation=cv2.INTER_AREA)

img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

custom_config = r'-l eng --oem 3 --psm 6 -c tessedit_char_whitelist="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789-:.$%./@& *"'
d = pytesseract.image_to_data(img_rgb, config=custom_config, output_type=Output.DICT)
df = pd.DataFrame(d)

# clean up blanks
df1 = df[(df.conf != '-1') & (df.text != ' ') & (df.text != '')]
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# sort blocks vertically
sorted_blocks = df1.groupby('block_num').first().sort_values('top').index.tolist()
for block in sorted_blocks:
    curr = df1[df1['block_num'] == block]
    sel = curr[curr.text.str.len() > 3]
    # sel = curr
    char_w = (sel.width / sel.text.str.len()).mean()
    prev_par, prev_line, prev_left = 0, 0, 0
    text = ''
    for ix, ln in curr.iterrows():
        # add new line when necessary
        if prev_par != ln['par_num']:
            text += '\n'
            prev_par = ln['par_num']
            prev_line = ln['line_num']
            prev_left = 0
        elif prev_line != ln['line_num']:
            text += '\n'
            prev_line = ln['line_num']
            prev_left = 0

        added = 0  # num of spaces that should be added
        if ln['left'] / char_w > prev_left + 1:
            added = int((ln['left']) / char_w) - prev_left
            text += ' ' * added
        text += ln['text'] + ' '
        prev_left += len(ln['text']) + added + 1
    text += '\n'
    print(text)


         A         B       c       D       E 
  1  LastName  Sales    Country Quarter 
  2  smith    1$16.753.00 uk    Qtr3 
  3 Johnson    $14.808.00 USA   Qtr 4 
  4 Williams    $10.644.00 UK   Qtr2 
  5 Jones       $1390.00 USA    tr3 
  6  Brown      $4865.00 USA    tra 
  7 Williams   $12.438.00 UK    Qtr. 
  8 Johnson     $9339.00 UK     Qtr2 
  9 smith      $18919.00 USA    Qtr3 
  10 Jones      $9213.00 USA    Qtr 
  11 Jones      $7/433.00 UK    Qtr 
  12/Brown      $3255.00 USA    Qtr2 
  13 Williams  $14.867.00 USA   Qtr3 
  14 Williams  $19302.00 UK     atr4 
  15 /Smith     $9.698.00 USA   Qtr 
  16 



In [4]:
#Load the output file to csv
with open('input.csv','w') as f: 
    f.write(text)

In [5]:
#Split the values and insert as rows and columns
x=pd.read_csv('input.csv')
x.head()

,A B c D E
0,1 LastName Sales Country Quarter
1,2 smith 1$16.753.00 uk Qtr3
2,3 Johnson $14.808.00 USA Qtr 4
3,4 Williams $10.644.00 UK Qtr2
4,5 Jones $1390.00 USA tr3


In [6]:
c=pd.DataFrame([' '.join(r).split() for r in x.values])
c

,0,1,2,3,4,5
0,1,LastName,Sales,Country,Quarter,None
1,2,smith,1$16.753.00,uk,Qtr3,None
2,3,Johnson,$14.808.00,USA,Qtr,4
3,4,Williams,$10.644.00,UK,Qtr2,None
4,5,Jones,$1390.00,USA,tr3,None
5,6,Brown,$4865.00,USA,tra,None
6,7,Williams,$12.438.00,UK,Qtr.,None
7,8,Johnson,$9339.00,UK,Qtr2,None
8,9,smith,$18919.00,USA,Qtr3,None
9,10,Jones,$9213.00,USA,Qtr,None


In [7]:
c.to_csv('Final.csv')